In [9]:
import baostock as bs
import pandas as pd
bs.login()
rs = bs.query_history_k_data_plus("sh.600000",
    "date,code,open,high,low,close,volume,amount,adjustflag",
    start_date='1999-11-10', end_date='2020-07-10', 
    frequency="d", adjustflag="2") 
#frequency="d"取日k线，adjustflag="3"默认不复权
print('query_history_k_data_plus respond error_code:' + rs.error_code)
print('query_history_k_data_plus respond  error_msg:' + rs.error_msg)

#### 打印结果集 ####
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)
print(result)

login success!
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
            date       code           open           high            low  \
0     1999-11-10  sh.600000   3.0164635000   3.0471394000   2.7608310000   
1     1999-11-11  sh.600000   2.8201377400   2.9019401400   2.8150250900   
2     1999-11-12  sh.600000   2.8487685800   2.8937599000   2.8395658100   
3     1999-11-15  sh.600000   2.8835346000   2.8886472500   2.8324081000   
4     1999-11-16  sh.600000   2.8508136400   2.8600164100   2.7076594400   
...          ...        ...            ...            ...            ...   
5002  2020-07-06  sh.600000  11.3000000000  12.3100000000  11.2900000000   
5003  2020-07-07  sh.600000  12.4500000000  12.6900000000  12.1000000000   
5004  2020-07-08  sh.600000  12.1700000000  12.4400000000  12.0300000000   
5005  2020-07-09  sh.600000  12.2600000000  12.2600000000  11.9400000000   
5006  2020-07-10  sh.600000  11.9800000000  12.09

In [245]:
import datetime
s = datetime.datetime.now().strftime('%Y-%m-%d')
s>'2010-10-22'

True

In [293]:
import baostock as bs
import pandas as pd
import datetime

STOCK_URL = '../static/data/'
class BaoStock(object):
    def __init__(self, code='sh.600000', start_date='2006-01-01', end_date=''):
        self.code = code
        self.start_date = start_date
        if end_date == '':
            self.end_date = datetime.datetime.now().strftime('%Y-%m-%d')
        else:
            self.end_date = end_date

    @staticmethod
    def today_str():
        return datetime.datetime.now().strftime('%Y-%m-%d')

    @staticmethod
    def today_date():
        return datetime.datetime.strptime(datetime.datetime.today().strftime("%Y-%m-%d"), '%Y-%m-%d').date()

    def __get_last_date(self, freq='day/'):
        date_data = pd.read_csv(STOCK_URL + freq + self.code + '.csv', usecols=['date'])
        print('读取文件{}'.format(self.code + '.csv'))
        last_date = date_data.iloc[-1, 0]
        print(last_date)
        return datetime.datetime.strptime(last_date, "%Y-%m-%d")

    def __result_show(self, freq='day/'):
        print('{} 存储完成，已更新至{}'.format((self.code + '.csv'), self.__get_last_date(freq=freq)))
        # TODO Log记录

    def __get_date_day(self):
        # 登陆系统
        lg = bs.login()
        # 显示登陆返回信息
        print('login respond error_code:' + lg.error_code)
        print('login respond  error_msg:' + lg.error_msg)

        # 获取沪深A股历史K线数据
        # 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。
        # 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
        rs = bs.query_history_k_data_plus(self.code,
                                          'date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST',
                                          start_date=self.start_date, end_date=self.end_date,
                                          frequency='d', adjustflag='3')
        print('query_history_k_data_plus respond error_code:' + rs.error_code)
        print('query_history_k_data_plus respond  error_msg:' + rs.error_msg)

        # 打印结果集
        data_list = []
        while (rs.error_code == '0') & rs.next():
            # 获取一条记录，将记录合并在一起
            data_list.append(rs.get_row_data())
        result = pd.DataFrame(data_list, columns=rs.fields)
        # 登出系统
        bs.logout()
        return result

    def __get_date_minute(self, frequency='5'):
        # 登陆系统
        lg = bs.login()
        # 显示登陆返回信息
        print('login respond error_code:' + lg.error_code)
        print('login respond  error_msg:' + lg.error_msg)

        # 获取沪深A股历史K线数据
        # 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。
        # 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
        rs = bs.query_history_k_data_plus(self.code,
                                          'date,time,code,open,high,low,close,volume,amount,adjustflag',
                                          start_date=self.start_date, end_date=self.end_date,
                                          frequency=frequency, adjustflag='3')
        print('query_history_k_data_plus respond error_code:' + rs.error_code)
        print('query_history_k_data_plus respond  error_msg:' + rs.error_msg)

        # 打印结果集
        data_list = []
        while (rs.error_code == '0') & rs.next():
            # 获取一条记录，将记录合并在一起
            data_list.append(rs.get_row_data())
        result = pd.DataFrame(data_list, columns=rs.fields)
        # 登出系统
        # TODO 进度条
        bs.logout()
        return result

    def init_stock_day(self):
        result = self.__get_date_day()
        # 本地化存储CSV
        result.to_csv(STOCK_URL + 'day/' + self.code + '.csv', mode='w', index=False)
        self.__result_show(freq='day/')

    def init_stock_minute(self):
        result = self.__get_date_minute()
        # 本地化存储CSV
        result.to_csv(STOCK_URL + 'min/' + self.code + '.csv', mode='w', index=False)
        self.__result_show(freq='min/')

    def up_to_date_day(self):
        url = 'day/'
        try:
            last_date = self.__get_last_date(freq=url)
            print('读取文件{}'.format(self.code + '.csv'))
            # TODO 时间比较BUG
            if last_date.strftime("%Y-%m-%d") == self.today_str:
                print('当前数据已经最新')
            else:
                self.start_date = (last_date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
                self.end_date = datetime.datetime.now().strftime('%Y-%m-%d')
                result = self.__get_date_day()
                # 向CSV末尾追加数据
                result.to_csv(STOCK_URL + url + self.code + '.csv', mode='a', header=False, index=False)
                self.__result_show(freq=url)
        except Exception as e:
            print('未找到对应CSV文件')
            self.start_date = '2006-01-01'
            self.end_date = datetime.datetime.now().strftime('%Y-%m-%d')
            self.init_stock_day()

    def up_to_date_minute(self):
        url = 'min/'
        try:
            last_date = self.__get_last_date(freq=url)
            print('读取文件{}'.format(self.code + '.csv'))
            # TODO 时间比较BUG
            if last_date.strftime("%Y-%m-%d") == self.today_str():
                print('当前数据已经最新')
            else:
                self.start_date = (last_date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
                self.end_date = datetime.datetime.now().strftime('%Y-%m-%d')
                result = self.__get_date_minute()
                # 向CSV末尾追加数据
                result.to_csv(STOCK_URL + url + self.code + '.csv', mode='a', header=False, index=False)
                self.__result_show(freq=url)
        except Exception as e:
            print('未找到对应CSV文件')
            self.start_date = '2006-01-01'
            self.end_date = datetime.datetime.now().strftime('%Y-%m-%d')
            self.init_stock_minute()

    @classmethod
    def get_all_stock(cls):
        # 登陆系统
        lg = bs.login()
        # 显示登陆返回信息
        print('login respond error_code:' + lg.error_code)
        print('login respond  error_msg:' + lg.error_msg)

        # 获取证券信息
        t = cls.today_str
        rs = bs.query_all_stock(day='2020-06-01')
        print('query_all_stock respond error_code:' + rs.error_code)
        print('query_all_stock respond  error_msg:' + rs.error_msg)

        # 打印结果集
        data_list = []
        while (rs.error_code == '0') & rs.next():
            # 获取一条记录，将记录合并在一起
            data_list.append(rs.get_row_data())
        result = pd.DataFrame(data_list, columns=rs.fields)
        # 结果集输出到csv文件
        result.to_csv(STOCK_URL + 'all_stock.csv', mode='w', index=False, encoding="gbk")

        # 登出系统
        bs.logout()

    
t = BaoStock(code='sh.600011', end_date='2007-01-01')
t.up_to_date_minute()
# t.today_str()


未找到对应CSV文件
login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
读取文件sh.600011.csv
2020-06-22
sh.600011.csv 存储完成，已更新至2020-06-22 00:00:00


In [206]:
import datetime
d1 = datetime.datetime.today()
d2 = datetime.datetime.strptime('2020-08-09', "%Y-%m-%d")

if d1>d2:
    print('D1大')
else:
    print('D2大')

D2大


In [267]:
s = datetime.datetime.now().strftime('%Y-%m-%d')
type(s)

a= datetime.datetime.strptime('2020-06-22', "%Y-%m-%d").strftime("%Y-%m-%d")
type(a)

a==s

True

In [278]:
t = pd.read_csv(STOCK_URL + 'min/' + 'sh.600081' + '.csv', usecols=['date'])
temp = t.iloc[-1, 0]
type(temp)
temp <= datetime.datetime.now().strftime('%Y-%m-%d')

True

In [290]:
datetime.datetime.now().strftime('%Y-%m-%d')

'2020-06-22'

In [19]:
import baostock as bs
import pandas as pd
import datetime

STOCK_URL = '../static/data/'
class BaoStock(object):
    def __init__(self, code='sh.600000', start_date='2006-01-01', end_date=''):
        self.code = code
        self.start_date = start_date
        if end_date == '':
            self.end_date = datetime.datetime.now().strftime('%Y-%m-%d')
        else:
            self.end_date = end_date

    @staticmethod
    def today_str():
        return datetime.datetime.now().strftime('%Y-%m-%d')

    @staticmethod
    def today_date():
        return datetime.datetime.strptime(datetime.datetime.today().strftime("%Y-%m-%d"), '%Y-%m-%d').date()

    def __get_last_date(self, freq='day/'):
        date_data = pd.read_csv(STOCK_URL + freq + self.code + '.csv', usecols=['date'])
        print('读取文件{}'.format(self.code + '.csv'))
        last_date = date_data.iloc[-1, 0]
        print(last_date)
        return datetime.datetime.strptime(last_date, "%Y-%m-%d")

    def __result_show(self, freq='day/'):
        print('{} 存储完成，已更新至{}'.format((self.code + '.csv'), self.__get_last_date(freq=freq)))
        # TODO Log记录

    def __get_date_day(self):
        # 登陆系统
        lg = bs.login()
        # 显示登陆返回信息
        print('login respond error_code:' + lg.error_code)
        print('login respond  error_msg:' + lg.error_msg)

        # 获取沪深A股历史K线数据
        # 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。
        # 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
        rs = bs.query_history_k_data_plus(self.code,
                                          'date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST',
                                          start_date=self.start_date, end_date=self.end_date,
                                          frequency='d', adjustflag='3')
        print('query_history_k_data_plus respond error_code:' + rs.error_code)
        print('query_history_k_data_plus respond  error_msg:' + rs.error_msg)

        # 打印结果集
        data_list = []
        while (rs.error_code == '0') & rs.next():
            # 获取一条记录，将记录合并在一起
            data_list.append(rs.get_row_data())
        result = pd.DataFrame(data_list, columns=rs.fields)
        # 登出系统
        bs.logout()
        return result

    def __get_date_minute(self, frequency='5'):
        # 登陆系统
        lg = bs.login()
        # 显示登陆返回信息
        print('login respond error_code:' + lg.error_code)
        print('login respond  error_msg:' + lg.error_msg)

        # 获取沪深A股历史K线数据
        # 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。
        # 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
        rs = bs.query_history_k_data_plus(self.code,
                                          'date,time,code,open,high,low,close,volume,amount,adjustflag',
                                          start_date=self.start_date, end_date=self.end_date,
                                          frequency=frequency, adjustflag='3')
        print('query_history_k_data_plus respond error_code:' + rs.error_code)
        print('query_history_k_data_plus respond  error_msg:' + rs.error_msg)

        # 打印结果集
        data_list = []
        while (rs.error_code == '0') & rs.next():
            # 获取一条记录，将记录合并在一起
            data_list.append(rs.get_row_data())
        result = pd.DataFrame(data_list, columns=rs.fields)
        # 登出系统
        # TODO 进度条
        bs.logout()
        return result

    def init_stock_day(self):
        result = self.__get_date_day()
        # 本地化存储CSV
        result.to_csv(STOCK_URL + 'day/' + self.code + '.csv', mode='w', index=False)
        self.__result_show(freq='day/')

    def init_stock_minute(self):
        result = self.__get_date_minute()
        # 本地化存储CSV
        result.to_csv(STOCK_URL + 'min/' + self.code + '.csv', mode='w', index=False)
#         self.__result_show(freq='min/')

    def up_to_date_day(self):
        url = 'day/'
        try:
            last_date = self.__get_last_date(freq=url)
            print('读取文件{}'.format(self.code + '.csv'))
            # TODO 时间比较BUG
            if last_date.strftime("%Y-%m-%d") == self.today_str:
                print('当前数据已经最新')
            else:
                self.start_date = (last_date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
                self.end_date = datetime.datetime.now().strftime('%Y-%m-%d')
                result = self.__get_date_day()
                # 向CSV末尾追加数据
                result.to_csv(STOCK_URL + url + self.code + '.csv', mode='a', header=False, index=False)
                self.__result_show(freq=url)
        except Exception as e:
            print('未找到对应CSV文件')
            self.start_date = '2006-01-01'
            self.end_date = datetime.datetime.now().strftime('%Y-%m-%d')
            self.init_stock_day()

    def up_to_date_minute(self):
        url = 'min/'
        try:
            last_date = self.__get_last_date(freq=url)
            print('读取文件{}'.format(self.code + '.csv'))
            # TODO 时间比较BUG
            if last_date.strftime("%Y-%m-%d") == self.today_str():
                print('当前数据已经最新')
            else:
                self.start_date = (last_date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
                self.end_date = datetime.datetime.now().strftime('%Y-%m-%d')
                result = self.__get_date_minute()
                # 向CSV末尾追加数据
                result.to_csv(STOCK_URL + url + self.code + '.csv', mode='a', header=False, index=False)
                self.__result_show(freq=url)
        except Exception as e:
            print('未找到对应CSV文件')
            self.start_date = '2006-01-01'
            self.end_date = datetime.datetime.now().strftime('%Y-%m-%d')
            self.init_stock_minute()

    @classmethod
    def get_all_stock(cls):
        # 登陆系统
        lg = bs.login()
        # 显示登陆返回信息
        print('login respond error_code:' + lg.error_code)
        print('login respond  error_msg:' + lg.error_msg)

        # 获取证券信息
        t = cls.today_str
        rs = bs.query_all_stock(day='2020-06-01')
        print('query_all_stock respond error_code:' + rs.error_code)
        print('query_all_stock respond  error_msg:' + rs.error_msg)

        # 打印结果集
        data_list = []
        while (rs.error_code == '0') & rs.next():
            # 获取一条记录，将记录合并在一起
            data_list.append(rs.get_row_data())
        result = pd.DataFrame(data_list, columns=rs.fields)
        # 结果集输出到csv文件
        result.to_csv(STOCK_URL + 'all_stock.csv', mode='w', index=False, encoding="gbk")

        # 登出系统
        bs.logout()
        
t = BaoStock(code='sh.000001',end_date='2007-01-01')
t.init_stock_minute()


# code_data = pd.read_csv(STOCK_URL + 'all_stock.csv', usecols=['code'])
# import time
# for row in code_data.iterrows():
#     s = row[1].code
#     temp = BaoStock(code=s)
#     temp.up_to_date_day()
#     print('{} is done!'.format(s))
#     time.sleep(1)
    
# for row in code_data.iterrows():
#     s = row[1].code
#     temp = BaoStock(code=s)
#     temp.up_to_date_minute()
#     print('{} is done!'.format(s))
#     time.sleep(5)

login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
(0, 10)


In [3]:
import datetime
datetime.datetime.now().hour

1

In [ ]:
import baostock as bs
import pandas as pd
STOCK_URL = '../static/data/'
# code_data = pd.read_csv(STOCK_URL + 'all_stock.csv', usecols=['code'])
bs.login()
stock_rs = bs.query_all_stock('2006-06-23')
data_list = []
while (rs.error_code == '0') & rs.next():
    data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)
bs.logout()
print(result)

In [30]:
# import baostock as bs
# import pandas as pd

STOCK_URL = '../static/data/'
lg = bs.login()
rs = bs.query_history_k_data_plus('sh.600026','date,time,code,open,high,low,close,volume,amount,adjustflag',start_date='2006-01-01', end_date='2020-06-23',frequency='5', adjustflag='3')
data_list = []
while (rs.error_code == '0') & rs.next():
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)
%time result.to_csv(STOCK_URL + 'test.csv', mode='w', index=False, encoding="GBK")

login success!
CPU times: user 632 ms, sys: 24.5 ms, total: 656 ms
Wall time: 660 ms
